In [16]:
import os
import pandas as pd
import numpy as np
from django.db import transaction
from django.conf import settings
from intranet.academic.models import (
    SchoolYear,
    AcademicYear, 
    CourseType, 
    CourseGroup, 
    ManagementUnit,
    Curriculum,
    Concentration,
    Course,
    CourseRequisite
)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [17]:
# Import SchoolYear
schoolyear_data = pd.read_csv('academic_schoolyear.csv', header=0).to_dict(orient='records')
with transaction.atomic():
    school_years = []
    for items in schoolyear_data:
        school_years.append(SchoolYear(**items))
    
    SchoolYear.objects.bulk_create(school_years)

In [18]:
# Import AcadmicYear
academicyear_data = pd.read_csv('academic_academicyear.csv', header=0).to_dict(orient='records')
with transaction.atomic():
    academic_years = []
    for items in academicyear_data:
        academic_years.append(AcademicYear(**items))
    
    AcademicYear.objects.bulk_create(academic_years)

In [19]:
# Import CourseType
coursetype_data = pd.read_csv('academic_coursetype.csv', header=0).to_dict(orient='records')
with transaction.atomic():
    coursetypes = []
    for items in coursetype_data:
        coursetypes.append(CourseType(**items))
    
    CourseType.objects.bulk_create(coursetypes)

In [20]:
# Import CourseGroup
coursegroup_data = pd.read_csv('academic_coursegroup.csv', header=0).to_dict(orient='records')
with transaction.atomic():
    coursegroups = []
    for items in coursegroup_data:
        coursegroups.append(CourseGroup(**items))
    
    CourseGroup.objects.bulk_create(coursegroups)

In [21]:
# Import ManagementUnit
managementunit_data = pd.read_csv('academic_managementunit.csv', header=0).replace({np.nan: None}).to_dict(orient='records')
with transaction.atomic():
    units = []
    for items in managementunit_data:
        units.append(ManagementUnit(**items))
    
    ManagementUnit.objects.bulk_create(units)

In [22]:
# Import Concentration AKT
concentration_data = pd.read_csv('academic_concentration.csv', header=0).replace({np.nan: None}).to_dict(orient='records')
with transaction.atomic():
    concents = []
    for items in concentration_data:
        concents.append(Concentration(**items))
    
    Concentration.objects.bulk_create(concents)

In [23]:
# Import Course AKT
course_data = pd.read_csv('academic_course.csv', header=0).replace({np.nan: None}).to_dict(orient='records')
with transaction.atomic():
    courses = []
    for items in course_data:
        courses.append(Course(**items))
    
    Course.objects.bulk_create(courses)

In [24]:
# Import Course Requisite AKT
course_requisite_data = pd.read_csv('academic_course_requisites.csv', header=0).replace({np.nan: None}).to_dict(orient='records')
with transaction.atomic():
    reqs = []
    for items in course_requisite_data:
        course = Course.objects.get(old_code=items['course'])
        requisite = Course.objects.get(old_code=items['requisite'])
        reqs.append(CourseRequisite(course=course, requisite=requisite))
    
    CourseRequisite.objects.bulk_create(reqs)

In [25]:
# Import Curriculum
curriculum_data = pd.read_csv('academic_curriculum.csv', header=0).replace({np.nan: None}).to_dict(orient='records')
with transaction.atomic():
    curriculums = []
    for items in curriculum_data:
        curriculums.append(Curriculum(**items))
    
    Curriculum.objects.bulk_create(curriculums)

In [26]:
# Import Curricullum Course AKT
curriculumcourse_data = pd.read_csv('academic_curriculumcourse.csv', header=0).replace({np.nan: None}).to_dict(orient='records')
with transaction.atomic():
    for items in curriculumcourse_data:
        course = Course.objects.get(old_code=items['course'])
        curriculum_id = items['curriculum_id']
        semester = items['semester_number']
        curriculum_course = CurriculumCourse(course=course, curriculum_id=curriculum_id, semester_number=semester)
        curriculum_course.save()
        
        # Add concentration
        concents = items['concentration']
        if concents:
            for code in concents.split(','):
                concent = Concentration.objects.get(code=code)
                curriculum_course.concentration.add(concent)